In [199]:
import requests
from xml.etree import ElementTree
from collections import defaultdict
import numpy as np
import pandas as pd
import statistics
import math
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import json
import urllib.request
import escher
from escher import Builder
import cobra
import time
from time import sleep

In [200]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
                d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [201]:
def createTree(link):
    time.sleep(1)
    response = requests.get(link)
    tree = ElementTree.fromstring(response.content)
    return tree

In [202]:
def createDictFromName(complexList, errors):
    result = {}

    for name in complexList:
        try:
            linkComplex = "https://websvc.biocyc.org/getxml?id=ECOLI:" + name
            treeComplex = createTree(linkComplex)
            treeDictComplex = etree_to_dict(treeComplex)
            result[name] = treeDictComplex
        except Exception as e:
            errors.append((name, e))

    return result

In [203]:
def checkAndReplaceCommonName(reaction, names):
    e

In [204]:
def checkAndReplaceSynonyms(reactionList, names):
    gh

In [205]:
simData = np.load(r"../../out/geneRxnVerifData/output.npy", allow_pickle=True, encoding='ASCII')
fluxesWithCaption = simData.tolist()['agents']['0']['listeners']['fba_results']['estimated_fluxes']
complexes = simData.tolist()['agents']['0']['bulk']

metabolData = np.load(r"../../out/geneRxnVerifData/stoichiometry.npy", allow_pickle=True, encoding='ASCII')
rxn_metabolites = metabolData.tolist()

In [206]:
reactions = fluxesWithCaption.keys()

In [207]:
ecReactions = pd.read_csv('reactions.tsv', sep='\t', header=0)['Reaction'].tolist()

In [208]:
foundNames = []
notFound = []
for rxn in ecReactions:
    found = False
    for key in fluxesWithCaption:
        if rxn in key:
            found = True
            foundNames.append((rxn, key))
    if not found:
        notFound.append(rxn)

In [209]:
prev = ""
fluxes = {}
for names in foundNames:
    if prev == names[0]:
        for i in range(len(fluxes[prev])):
            if "(reverse)" in names[1]:
                fluxes[prev][i] -= fluxesWithCaption[names[1]][i]
            else:
                fluxes[prev][i] += fluxesWithCaption[names[1]][i]
    else:
        prev = names[0]
        fluxes[prev] = fluxesWithCaption[names[1]]

In [210]:
with open('../../iJO1366.Central metabolism.json') as json_file:
    data = json.load(json_file)
    
escherRxns = data[1]['reactions']
names = []

for number in escherRxns:
    reactionName = escherRxns[number]['name']
    names.append(reactionName)

In [211]:
errors = []
reactionDict = createDictFromName(fluxes.keys(), errors)

In [241]:
%store reactionDict

Stored 'reactionDict' (dict)


In [246]:
ECnames = []

for key in reactionDict.keys():
    dicte = reactionDict[key]
    try:
        commonName = dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']['common-name']['#text']
        try:
            synonyms = [x['#text'] for x in dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']['synonym']]
        except:
            synonyms = dicte['ptools-xml']['Reaction']['enzymatic-reaction']['Enzymatic-Reaction']['synonym']['#text']
        ECnames.append((commonName, synonyms))
    except Exception as e:
        errors.append((key, e))

[('FMN adenylyltransferase',
  ['FAD pyrophosphorylase', 'ATP:FMN adenylyltransferase', 'FAD synthetase']),
 ('lipid kinase', 'phosphatidylglycerol kinase'),
 ('phosphatidylserine synthase',
  ['CDPdiacylglycerol-serine O-phosphatidyltransferase',
   'CDPdiglyceride-serine O-phosphatidyltransferase',
   'CDPdiacylglycerol:L-serine 3-O-phosphatidyltransferase']),
 ('CMP kinase',
  ['CK',
   'cytidine monophosphate kinase',
   'ATP:CMP phosphotransferase',
   'cytidylate kinase']),
 ('galactarate dehydratase',
  ['galactarate dehydrase', 'D-galactarate hydro-lyase', 'GalcD']),
 ('isoleucine transaminase',
  'branched chain amino acid:2-oxoglutarate aminotransferase'),
 ('uroporphyrinogen-III synthase',
  ['uroporphyrinogen III cosynthase',
   'hydroxymethylbilane hydro-lyase (cyclizing)']),
 ('tRNA Um32 methyltransferase', 'TrMet(Um32)'),
 ('inositol-phosphate phosphatase',
  ['<i>myo</i>-inositol-1(or 4)-monophosphatase',
   '<i>myo</i>-inositol-1(or 4)-phosphate phosphohydrolase',
   '